In [1]:
import os

import rasterio
import numpy as np


In [2]:
# directory where the earth engine result was downloaded to from drive
data_dir = '/Users/d3y010/projects/cerf/data/srtm'

# directory to store technology specific suitability layers in
tech_dir = '/Users/d3y010/projects/cerf/suitability/data/technology_specific'

# downloaded file
srtm_epsg6380 = os.path.join(data_dir, 'SRTM90_V4_slope_percent_1km_epsg6350.tiff')

# reprojected and masked file to CERF's CRS and extent
srtm_albers = os.path.join(data_dir, 'SRTM90_V4_slope_percent_1km_albers.tif')

# land mask raster; 0 (suitable), 1 (unsuitable)
land_mask = '/Users/d3y010/projects/cerf/suitability/data/reference/cerf_landmask.tif'

# exclusion layer for wind where >20 percent slope is unsuitable
cerf_slope_20per = os.path.join(tech_dir, 'cerf_srtm_slope_20pct.tif')

# exclusion layer for common where >12 percent slope is unsuitable
cerf_slope_12per = os.path.join(tech_dir, 'cerf_srtm_slope_12pct.tif')

cerf_slope_10per = os.path.join(tech_dir, 'cerf_srtm_slope_10pct.tif')
cerf_slope_5per = os.path.join(tech_dir, 'cerf_srtm_slope_5pct.tif')


### Generate the following slope raster from SRTM 90m data via Google Earth Engine

```javascript

// get sample area to extract|
var geometry = 
    ee.Geometry.Polygon(
        [[[-128.59843750000002, 50.17140850495461],
          [-128.59843750000002, 23.55613021145419],
          [-64.78984375000002, 23.55613021145419],
          [-64.78984375000002, 50.17140850495461]]], null, false);

// DEM
var dataset = ee.Image('CGIAR/SRTM90_V4').clip(geometry);

// extract elevation data
var elevation = dataset.select('elevation');

// calculate slope in degrees
var slope = ee.Terrain.slope(elevation);

// convert to percent slope
var slope_percent = slope.divide(100).multiply(Math.PI).tan().multiply(100).rename('Percent');

// reproject into NAD83 Albers in meters; CERF's standard ESRI:102003 is not available
var percent_albers = slope_percent.reproject({
  crs: 'EPSG:6350',
  scale: 1000
});

// export the result to my drive
Export.image.toDrive({
  image: percent_albers,
  description: 'SRTM90_V4_slope_percent_1km_epsg6350',
  folder: 'earth_engine',
  region: geometry,
  scale: 1000,
  crs: 'EPSG:6350'
});
```

### Reproject to CERF's CRS and extent

In [3]:
# gdal warp command
gdal_cmd = f'gdalwarp -s_srs EPSG:6350 -t_srs ESRI:102003 -tr 1000.0 1000.0 -r bilinear -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -te_srs ESRI:102003 -multi -of GTiff {srtm_epsg6380} {srtm_albers}'

os.system(gdal_cmd)


0

### Reclassify to less than or equal to 20 percent slope as 0 (suitable) and above 1 (unsuitable) 

In [4]:
with rasterio.open(land_mask) as msk:
    mask_arr = msk.read(1)
    mask_arr = np.where(mask_arr == 0, np.nan, mask_arr)


In [23]:

# create 20 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 20 percent unsuitable
    arr = np.where(arr > 20, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(cerf_slope_20per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)
        

# create 12 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 20 percent unsuitable
    arr = np.where(arr > 12, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(cerf_slope_12per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)



In [5]:
# create 10 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 10 percent unsuitable
    arr = np.where(arr > 10, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(cerf_slope_10per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)


In [6]:
# create 5 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 5 percent unsuitable
    arr = np.where(arr > 5, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(cerf_slope_5per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)
        